# ***README***
리딩/저장이 안되는 경우는 크게 2가지:
*   호환되지 않는 타입(Non-compatible type.)
*   메모리 아웃(Out of memory; suggest to convert models that smaller than 6GB only.) 

호환되지 않는 모델은 거의 없지만 메모리 아웃이 치명적이니까 가급적이면 RAM을 늘리거나 6GB 이하의 모델만 변환을 권장.


In [1]:
#@title Prepare
check=int(input('구글 드라이브를 사용하시겠습니까? 1: 예 0: 아니오 '))

if check==1:
    from google.colab import drive
    drive.mount('/content/drive')
elif check!=1:
    pass

!pip install torch safetensors pytorch-lightning
!sudo apt install aria2

from google.colab import output
output.clear()

if check==1:
    print('(권장) 구글 드라이브를 임포트했습니다.')
elif check!=1:
    print('구글 드라이브를 임포트하지 않았습니다.')

(권장) 구글 드라이브를 임포트했습니다.


In [ ]:
#@title Download
!aria2c --continue --allow-overwrite --always-resume --summary-interval=10 --disk-cache=64M --min-split-size=8M --max-concurrent-downloads=8 --max-connection-per-server=8 --max-overall-download-limit=0 --max-download-limit=0 --split=8 

In [ ]:
#@title Universal Converter
import os
import torch
import pytorch_lightning
from safetensors.torch import save_file

files = os.listdir()
for f in files:
    if f.lower().endswith('.ckpt'):
        print(f'Loading {f}...')
        fn = f"{f.replace('.ckpt', '')}.safetensors"

        if fn in files:
            print(f'Skipping, as {fn} already exists.')
            continue

        try:
            with torch.no_grad():
                weights = torch.load(f)["state_dict"]
                weights.pop("state_dict")
                fn = f"{f.replace('.ckpt', '')}.safetensors"
                print(f'Saving {fn}...')
                save_file(weights, fn)
        except Exception as ex:
            print(f'ERROR converting {f}: {ex}; Trying another method')
            with torch.no_grad():
                weightsx=torch.load(f)
                fn = f"{f.replace('.ckpt', '')}.safetensors"
                print(f'Saving {fn}...')
                save_file(weightsx, fn)

print('Done!')

In [2]:
#@title Save to Google Drive
!cp /content/drive/MyDrive/StableDiffusion/models/Stable-diffusion

#multi file saving은 !cp (파일1).safetensors (파일2).safetensors (파일3).safetensors (구글 드라이브 내 저장 경로) 식으로 입력할 것.